In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas as pd
import numpy as np
from tensorflow.keras.layers import LSTM, Dense, Flatten, Dropout, Dropout
from tensorflow.keras.models import Sequential
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import numpy as np
import random
import os

In [2]:
from IPython.core.display import display, HTML

display(HTML("<style>.container {width:100% !important; }</style>"))

pd.set_option('display.max_columns', 500)

In [9]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
seed_everything(42)

In [3]:
df = pd.read_excel('./data/dataset.xlsx')

In [4]:
df = df[['STDR_YM', 'CTPRVN_NM', 'SIGNGU_NM', 'LTRS_CO', '중견기업수', '중소기업수', 'TOT_ENTRPRS_CO', 'ECNY_CO',
       'RETIRE_CO', '지급액', '수출건수', '수출금액', '수입건수', '수입금액', '무역수지',
       '취득자수', '상실자수', '수급자격신청자수', '수급자격인정자수', '실업급여지급자수', '실업급여지급건수',
       '실업급여지급액', '신규성립사업장수', '소멸사업장수', 'elec_use', 'EMPLY_CO', '피보험자수']]

In [6]:
df.drop('EMPLY_CO', axis=1, inplace=True)

In [10]:
window_size = 6
features = 23
cnt = 0

In [14]:
sd = df['CTPRVN_NM'].unique()

for i, v in enumerate(sd):
    tmp = df[df['CTPRVN_NM']==v]
    for j, vv in enumerate(tmp['SIGNGU_NM'].unique()):
        tmp2 = tmp[tmp['SIGNGU_NM']==vv].drop(['CTPRVN_NM','SIGNGU_NM'], axis=1)
        tmp2.set_index('STDR_YM', inplace=True)
        if cnt == 0:
            one_data = tmp2.values
        else:
            try:
                one_data = np.append(one_data, tmp2.values.reshape(-1, len(tmp2), features), axis=0).reshape(-1, len(tmp2), features)
            except:
                one_data = np.append(one_data, tmp2.values, axis=0).reshape(-1, len(tmp2), features)
        cnt += 1 

In [16]:
one_data.shape

(231, 79, 23)

In [19]:
# sc = MinMaxScaler()
# for i in range(one_data.shape[1]):
#     sc.partial_fit(one_data[:, i, :])

# sc_results = []
# for i in range(one_data.shape[1]):
#     sc_results.append(sc.transform(one_data[:, i, :]).reshape(one_data.shape[0], 1, one_data.shape[2]))
    
# sc_od = np.concatenate(sc_results, axis=1)

In [23]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [32]:
# create a differenced series
def difference(dataset, interval=1):
	diff = list()
	for i in range(interval, len(dataset)):
		value = dataset[i] - dataset[i - interval]
		diff.append(value)
	return pd.Series(diff)

In [41]:
# transform series into train and test sets for supervised learning
def prepare_data(series, n_test, n_lag, n_seq):
	# extract raw values
	raw_values = series.values
	# transform data to be stationary
	diff_series = difference(raw_values, 1)
	diff_values = diff_series.values
	diff_values = diff_values.reshape(len(diff_values), 1)
	# rescale values to -1, 1
	scaler = MinMaxScaler(feature_range=(-1, 1)); print(diff_values.shape)
	scaled_values = scaler.fit_transform(diff_values)
	scaled_values = scaled_values.reshape(len(scaled_values), 1)
	# transform into supervised learning problem X, y
	supervised = series_to_supervised(scaled_values, n_lag, n_seq)
	supervised_values = supervised.values
	# split into train and test sets
	train, test = supervised_values[0:-n_test], supervised_values[-n_test:]
	return scaler, train, test

### 시작

In [34]:
n_lag = 6
n_seq = 1
n_test = 12

In [35]:
series = pd.DataFrame(one_data[0])

In [42]:
sc, train, test = prepare_data(series, n_test, n_lag, n_seq)

(78, 1)


ValueError: setting an array element with a sequence.